In [ ]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user input
user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")

# Function definition
function_def = [
    {
        "name": "generate_landing_surface_layout",
        #"description": "Generate a structured TLOF (Touchdown and Lift-Off Area) JSON layout with visual and dimensional properties. Defaults will be applied where values are missing.",
        "description": "Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area), including visual, geometric, lighting, and safety parameters. The layout should support customization for shape (Polygon, Circle, Rectangle), aircraft type, size, elevation, markings (solid or dashed), and landing marker (capital letter 'H' or 'V'). If any field is not specified, apply reasonable defaults. Note: 'markingType' accepts only 'solid' or 'dashed'. 'landingMarker' accepts only 'H' or 'V'.",
        "parameters": {
            "type": "object",
            "properties": {
                # Others
                "aircraft": {"type": "string"},
                "diameter": {"type": "number"},
                "shape_type": {
                    "type": "string",
                    "enum": ["Circle", "Polygon", "Rectangle"]
                },
                "position": {
                    "type": "array",
                    "items": {"type": "number"},
                    "minItems": 2,
                    "maxItems": 2
                },
                
                # Shape
                "sides": {
                    "type": "number",
                    "description": "Number of sides for polygonal TLOF shapes (minimum 4, default is 4). Used only if shapeType is 'Polygon'."
                },
                "surface_width": {
                    "type": "number",
                    "description": "Width of the TLOF surface in meters."
                },
                "surface_length": {
                    "type": "number",
                    "description": "Length of the TLOF surface in meters."
                },
                "surface_height": {
                    "type": "number",
                    "description": "Thickness of the TLOF platform in meters."
                },
                "rotation": {
                    "type": "number",
                    "description": "Rotation angle of the TLOF surface in degrees (0–359)."
                },
                "transparency": {
                    "type": "number",
                    "description": "Transparency of the TLOF surface. 0 is fully transparent, 1 is fully opaque."
                },
                "elevation": {
                    "type": "number",
                    "description": "Base Height also called as Base elevation in meters."
                },

                # Marking
                "markingType": {
                    "type": "string",
                    "description": "Type of marking pattern. Allowed values: 'solid' or 'dashed'. Defaults to 'dashed' if an unsupported value is provided.",
                    "enum": ["solid", "dashed"]
                },               
                "markingColor": {
                    "type": "string",
                    "description": "Color of the landing marker. Accepts only: white, yellow, blue, red, green, black, purple, orange, gray, brown. Warn message that no other color are accepted",
                    "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]
                },
                "markingThickness": {
                    "type": "number",
                    "description": "Thickness of the marking. Valid range: 0.1 to 1.5."
                },
                "dashLength": {
                    "type": "number",
                    "description": "Length of each dash (applies only if markingType is 'dashed'). Valid range: 0.5 to 5."
                },
                "dashDistance": {
                    "type": "number",
                    "description": "Distance between dashes (applies only if markingType is 'dashed'). Valid range: 0.5 to 5."
                },

                # Landing Marker
                 "landingMarker": {
                    "type": "string",
                    "description": "Capital letter for the landing marker. Accepts only 'H' or 'V'."
                },
                "markerScale": {
                    "type": "number",
                    "description": "Scale of the landing marker. Typical range: 0.5 to 50."
                },
                "markerThickness": {
                    "type": "number",
                    "description": "Extrusion thickness of the landing marker (in meters)."
                },
                "markerRotation": {
                    "type": "number",
                    "description": "Rotation angle of the landing marker (0–359°)."
                },
                "markerColor": {
                    "type": "string",
                    "description": "Color of the landing marker. Accepts only: white, yellow, blue, red, green, black, purple, orange, gray, brown.",
                    "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]
                },
                "letterThickness": {
                    "type": "number",
                    "description": "Line thickness of the marker letter."
                },

                # TDPC Marking
                "tdpcCategory": {
                    "type": "boolean",
                    "description": "Enable TDPC (Touchdown Positioning Circle) marking. This should be set to true only if any TDPC-related property (type, scale, thickness, extrusion, rotation, or color) is explicitly provided or modified. Otherwise, keep it false by default."
                },
                "tdpcType": {
                    "type": "string",
                    "description": "Shape type of the TDPC. Recommended: 'circle', 'cross', or 'square'.",
                    "enum": ["circle", "cross", "square"]
                },
                "tdpcScale": {
                    "type": "number",
                    "description": "Scale or radius of the TDPC. Valid range: 0.5 to 50."
                },
                "tdpcThickness": {
                    "type": "number",
                    "description": "Line thickness of the TDPC. Valid range: 0.1 to 50."
                },
                "tdpcExtrusion": {
                    "type": "number",
                    "description": "Extrusion thickness of the TDPC for 3D appearance. Valid range: 0 to 1."
                },
                "tdpcRotation": {
                    "type": "number",
                    "description": "Rotation angle of the TDPC in degrees (0–359)."
                },
                "tdpcColor": {
                    "type": "string",
                    "description": "Color of the TDPC. Must be one of the supported colors.",
                    "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]
                },
                # Lighting
                "lightColor": {
                    "type": "string",
                    "description": "Color of the TLOF lights. Accepts only: white, yellow, blue, red, green, black, purple, orange, gray, brown."
                },
                "lightScale": {
                    "type": "number",
                    "description": "Scale factor for the size of the light markers. Typical range: -20 to 100."
                },
                "lightDistance": {
                    "type": "number",
                    "description": "Distance between each light placed along the TLOF boundary, in meters. Valid range: 1 to 50."
                },
                "lightRadius": {
                    "type": "number",
                    "description": "Radius of individual lights on the TLOF in meters. Valid range: 0.1 to 1."
                },
                "lightHeight": {
                    "type": "number",
                    "description": "Height of the lights from the surface of the TLOF in meters. Valid range: 0.1 to 0.25."
                },
                
                # Safety Area
                "safetyAreaType": {
                    "type": "string",
                    "description": "Defines the method used to calculate the safety area around the TLOF. Use 'multiplier' to scale based on dValue, or 'offset' to use a fixed distance.",
                    "enum": ["offset", "multiplier"]
                },
                "dValue": {
                    "type": "number",
                    "description": "D-Value for safety area sizing. Required when safetyAreaType is 'multiplier'. Typical range: 1 to 20 meters."
                },
                "multiplier": {
                    "type": "number",
                    "description": "Multiplication factor applied to dValue to calculate the safety area. Used only with safetyAreaType set to 'multiplier'. Typical range: 0.5 to 20."
                },
                "offsetDistance": {
                    "type": "number",
                    "description": "Fixed offset distance (in meters) around the TLOF used to define the safety area. Used only when safetyAreaType is 'offset'. Typical range: 1 to 50 meters."
                },

                # Safety Net
                "curveAngle": {
                    "type": "number",
                    "description": "Curve angle of the safety net boundary in degrees. Default is 45."
                },
                "netHeight": {
                    "type": "number",
                    "description": "Height of the safety net in meters. Default is 15."
                },
                "safetyNetTransparency": {
                    "type": "number",
                    "description": "Transparency of the safety net (0 to 1)."
                }
            },
            "required": [
                "aircraft", "surface_length", "surface_width", "surface_height",
                "sides", "diameter", "markingThickness", "rotation", "transparency",
                "shape_type", "position"
            ]
        }
    }
]

# Get AI response
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    messages=[{"role": "user", "content": user_input}],
    functions=function_def,
    function_call={"name": "generate_landing_surface_layout"}
)

# Parse function arguments
function_args = json.loads(response.choices[0].message.function_call.arguments)

# Marking : markingType
allowed_marking_types = {"solid", "dashed"}
marking_type = function_args.get("markingType", "dashed").lower()
if marking_type not in allowed_marking_types:
    print(f"⚠️ Unsupported markingType '{marking_type}', defaulting to 'dashed'.")
    marking_type = "dashed"

# Set dash values conditionally
if marking_type == "dashed":
    dash_length = max(0.5, min(function_args.get("dashLength", 1), 5))
    dash_distance = max(0.5, min(function_args.get("dashDistance", 1), 5))
else:
    # Solid line → set minimum placeholder values
    dash_length = 0.5
    dash_distance = 0.5

#Choose color
allowed_colors = {"white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"}

def get_valid_color(input_value, default_value, label):
    color = function_args.get(input_value, default_value).lower()
    if color not in allowed_colors:
        print(f"⚠️ Unsupported {label} '{color}', defaulting to '{default_value}'.")
        return default_value
    return color

# Usage
light_color = get_valid_color("lightColor", "white", "lightColor")
marking_color = get_valid_color("markingColor", "white", "markingColor")
marker_color = get_valid_color("markerColor", "white", "markerColor")
tdpc_color = get_valid_color("tdpcColor", "white", "tdpcColor")

# Validate landing marker
allowed_markers = {"H", "V"}
marker_value = function_args.get("landingMarker", "H").upper()
if marker_value not in allowed_markers:
    print(f"⚠️ Unsupported landingMarker '{marker_value}', defaulting to 'H'.")
    marker_value = "H"

# Validate tdpcType
allowed_tdpc_types = {"circle", "cross", "square"}
tdpc_type_value = function_args.get("tdpcType", "circle").lower()
if tdpc_type_value not in allowed_tdpc_types:
    print(f"⚠️ Unsupported tdpcType '{tdpc_type_value}', defaulting to 'circle'.")
    tdpc_type_value = "circle"

# Validate safetyAreaType
allowed_safety_area_types = {"offset", "multiplier"}
safety_area_type_value = function_args.get("safetyAreaType", "multiplier").lower()
if safety_area_type_value not in allowed_safety_area_types:
    print(f"⚠️ Unsupported safetyAreaType '{safety_area_type_value}', defaulting to 'multiplier'.")
    safety_area_type_value = "multiplier"

# if any variable is set then caterory is enabled by setting to true else false
def is_category_enabled(keys: list, args: dict) -> bool:
    """
    Enables category if any variable from `keys` exists in the input `args`.
    This does not check values or defaults — only presence.
    """
    return any(k in args for k in keys)

#Marking
marking_keys = [
    "markingType", "markingColor", "markingThickness",
    "dashLength", "dashDistance"
]

markings_category_enabled = is_category_enabled(marking_keys, function_args)

#Landing marker
landing_marker_keys = [
    "landingMarker", "markerScale", "markerThickness",
    "letterThickness", "markerRotation", "markerColor"
]

landing_marker_category_enabled = is_category_enabled(landing_marker_keys, function_args)

#TDPC
tdpc_keys = [
    "tdpcType", "tdpcScale", "tdpcThickness",
    "tdpcExtrusion", "tdpcRotation", "tdpcColor"
]

tdpc_category_enabled = is_category_enabled(tdpc_keys, function_args)

#Lighting
light_keys = [
    "lightColor", "lightScale", "lightDistance",
    "lightRadius", "lightHeight"
]

light_category_enabled = is_category_enabled(light_keys, function_args)

#Safety Area
safety_area_keys = [
    "safetyAreaType", "dValue", "multiplier", "offsetDistance"
]

safety_area_category_enabled = is_category_enabled(safety_area_keys, function_args)

#Safety Net
safety_net_keys = [
    "curveAngle", "netHeight", "safetyNetTransparency"
]

safety_net_category_enabled = is_category_enabled(safety_net_keys, function_args)

# ID
tlof_id = f"TLOF-{int(time.time() * 1000)}"

# Final JSON
tlof_json = {
    "TLOF": [
        {
            "position": function_args["position"],  # [Lng, Lat]
            "dimensions": {
                # Others
                "unit": "m",
                "aircraftCategory": False,
                "aircraft": function_args.get("aircraft"),
                "diameter": max(1.0, min(function_args.get("diameter", 30.0), 200.0)),
                "isVisible": True,
                "layerName": "Praveen_TLOF",
                "shapeType": function_args.get("shape_type", "Rectangle"),
                "scaleCategory": False,
                "textureScaleU": 1,
                "textureScaleV": 1,
                "safetyNetScaleU": 1,
                "safetyNetScaleV": 1,
                
                # Shape
                "sides": max(3, min(function_args.get("sides", 4), 70)),
                "width": max(30, min(function_args.get("surface_width", 30), 30)),
                "length": max(30, min(function_args.get("surface_length", 30), 30)),
                "height": max(0.01, min(function_args.get("surface_height", 0.01), 5)),
                "rotation": max(0, min(function_args.get("rotation", 0), 359)),
                "transparency": max(0, min(function_args.get("transparency", 1), 1)),
                "baseHeight": max(0, min(function_args.get("elevation", 0), 1000)),

                # Marking
                "markingsCategory": markings_category_enabled,
                "markingType": marking_type,
                "markingColor": marking_color,
                "markingThickness": max(0.1, min(function_args.get("markingThickness", 0.5), 1.5)),
                "dashDistance": max(0.5, min(function_args.get("dashDistance", 1), 5)),
                "dashLength": max(0.5, min(function_args.get("dashLength", 1), 5)),

                # Landing Marker
                "landingMarkerCategory": landing_marker_category_enabled,
                "landingMarker": marker_value,
                "markerScale": max(0.5, min(function_args.get("markerScale", 1), 50)),
                "markerThickness": max(0, min(function_args.get("markerThickness", 0.02), 1)),
                "markerRotation": max(0, min(function_args.get("markerRotation", 0), 359)),
                "markerColor": marker_color,
                "letterThickness": max(0, min(function_args.get("letterThickness", 0.5), 50)),

                # TDPC Marking
                "tdpcCategory": tdpc_category_enabled,
                "tdpcType": tdpc_type_value,
                "tdpcScale": max(0.5, min(function_args.get("tdpcScale", 5), 50)),
                "tdpcThickness": max(0.1, min(function_args.get("tdpcThickness", 0.5), 50)),
                "tdpcRotation": max(0, min(function_args.get("tdpcRotation", 0), 359)),
                "tdpcExtrusion": max(0, min(function_args.get("tdpcExtrusion", 0.02), 1)),
                "tdpcColor": tdpc_color,

                # Lighting
                "lightCategory": light_category_enabled,
                "lightColor": light_color,
                "lightScale": max(-20, min(function_args.get("lightScale", 1), 100)),
                "lightDistance": max(1, min(function_args.get("lightDistance", 1), 50)),
                "lightRadius": max(0.1, min(function_args.get("lightRadius", 0.3), 1)),
                "lightHeight": max(0.1, min(function_args.get("lightHeight", 0.2), 0.25)),

                # Safety Area - start from here
                "safetyAreaCategory": safety_area_category_enabled,
                "safetyAreaType": safety_area_type_value,
                "dValue": max(1, min(function_args.get("dValue", 10), 20)),
                "multiplier": max(0.5, min(function_args.get("multiplier", 1.5), 20)),
                "offsetDistance": max(0, min(function_args.get("offsetDistance", 3), 50)),

                # Safety Net
                "safetyNetCategory": safety_net_category_enabled,
                "curveAngle": max(0, min(function_args.get("curveAngle", 45), 90)),
                "netHeight": max(0, min(function_args.get("netHeight", 15), 100)),
                "safetyNetTransparency": max(0, min(function_args.get("safetyNetTransparency", 0.5), 1)),
                "safetyNetColor": "#FF0000"
            }
        }
    ]
}

# Print output
print(json.dumps(tlof_json, indent=2))